# Arquitectura RAG: Paso a Paso y Comparativa

### Dependencias y Variables de Entorno

In [1]:
import os
import fitz
import numpy as np
from dotenv import load_dotenv
from groq import Groq
from sentence_transformers import SentenceTransformer
from typing import List
import google.generativeai as genai
from openai import OpenAI

In [37]:
load_dotenv('/.env', override=True)

False

## Parte 1: Flujo RAG Detallado

#### Paso 1: Extracción de Texto

In [38]:
def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = r'C:\Users\danie\Downloads\prueba\1-s2.0-S0378517324005799-main.pdf'
document_text = extract_text_from_pdf(pdf_path)

print(f"Primeros 500 caracteres del documento: {document_text[:500]}...")

Primeros 500 caracteres del documento: International Journal of Pharmaceutics 660 (2024) 124345
Available online 15 June 2024
0378-5173/© 2024 The Authors. Published by Elsevier B.V. This is an open access article under the CC BY license (http://creativecommons.org/licenses/by/4.0/).
Review 
Nanotechnology in medicine revolutionizing drug delivery for cancer and 
viral infection treatments 
Emina Karahmet Sher a,*, Mirna Alebi´c g, Marijana Markovi´c Boras b,c, Emina Boˇskailo c, 
Esma Karahmet Farhat c,d, Alma Karahmet c, Bojan Pavl...


#### Paso 2: Fragmentación (Chunking)

In [39]:
def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

text_chunks = chunk_text(document_text)

print(f"Número de chunks: {len(text_chunks)}")
print(f"\nEjemplo de un chunk: \n {text_chunks[0]}...")

Número de chunks: 181

Ejemplo de un chunk: 
 International Journal of Pharmaceutics 660 (2024) 124345
Available online 15 June 2024
0378-5173/© 2024 The Authors. Published by Elsevier B.V. This is an open access article under the CC BY license (http://creativecommons.org/licenses/by/4.0/).
Review 
Nanotechnology in medicine revolutionizing drug delivery for cancer and 
viral infection treatments 
Emina Karahmet Sher a,*, Mirna Alebi´c g, Marijana Markovi´c Boras b,c, Emina Boˇskailo c, 
Esma Karahmet Farhat c,d, Alma Karahmet c, Bojan Pavlovi´c e, Farooq Sher a,**, Lana Leki´c f 
a School of Science and Technology, Nottingham Trent University, Nottingham NG11 8NS, United Kingdom 
b Department of Laboratory Diagnostic, University Clinical Hospital Mostar, Mostar 88000, Bosnia and Herzegovina 
c International Society of Engineering Science and Technology, Nottingham, United Kingdom 
d Department of Food and Nutrition, Faculty of Food Technology, Juraj Strossmayer University of Osijek, O

#### Paso 3: Creación de Embeddings

In [40]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def create_embeddings(texts: List[str]) -> np.ndarray:
    return embedding_model.encode(texts)

chunk_embeddings = create_embeddings(text_chunks)

print(f"Dimensiones de la matriz de embeddings: {chunk_embeddings.shape}")

Dimensiones de la matriz de embeddings: (181, 384)


#### Paso 4: Búsqueda Semántica

In [41]:
def _cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def semantic_search(query_embedding: np.ndarray, chunk_embeddings: np.ndarray, k: int = 3) -> List[int]:
    similarity_scores = [
        (i, _cosine_similarity(query_embedding, chunk_emb))
        for i, chunk_emb in enumerate(chunk_embeddings)
    ]
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    return [index for index, _ in similarity_scores[:k]]

user_query = "Realizame un resumen del documento"
query_embedding = create_embeddings([user_query])[0]
top_indices = semantic_search(query_embedding, chunk_embeddings)
relevant_chunks = [text_chunks[i] for i in top_indices]

print(f"Pregunta: {user_query}")
print(f"\nChunks más relevantes encontrados ({len(relevant_chunks)}):")
for i, chunk in enumerate(relevant_chunks):
    print(f"--- Chunk {i+1} \n {chunk[:300]}...")

Pregunta: Realizame un resumen del documento

Chunks más relevantes encontrados (3):
--- Chunk 1 
 nslational 
difficulties. 
CRediT authorship contribution statement 
Emina Karahmet Sher: Writing – review & editing, Writing – 
original draft, Supervision, Data curation, Conceptualization. Mirna 
Alebi´c: Writing – original draft, Software, Resources, Investigation. 
Marijana Markovi´c Boras: Wri...
--- Chunk 2 
 l entry, the 
conditions in which the implantation of the genome occurs in the host 
genome, as well as the conditions for transcription of viral structural and 
nonstructural proteins, and the replication and reconstitution of the 
virus, can vary significantly from virus to virus but always have s...
--- Chunk 3 
 er tissues. Striving for this goal, in the last 
10–20 years the use of nanotechnological solutions has surged for tar­
geted drug delivery. Basic research in this field continually develops new 
nanomaterials with improved capabilities, gradually integrating them 

#### Paso 5: Generación de Respuesta Aumentada

In [42]:
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def generate_augmented_response(user_query: str, relevant_chunks: List[str]) -> str:
    context_str = "\n\n".join(relevant_chunks)
    prompt_message = [
        {
            "role": "system",
            "content": "Eres un asistente que responde preguntas basándote únicamente en el contexto proporcionado. Si la respuesta no está en el contexto, di que no tienes suficiente información."
        },
        {
            "role": "user",
            "content": f"Contexto:\
{context_str}\\n\nPregunta: {user_query}"
        }
    ]
    
    chat_completion = groq_client.chat.completions.create(
        messages=prompt_message,
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

final_response = generate_augmented_response(user_query, relevant_chunks)

print(f"Respuesta generada: \n {final_response}")

Respuesta generada: 
 Basándome en el contexto proporcionado, puedo resumir el documento de la siguiente manera:

El documento se centra en la utilización de soluciones nanotecnológicas para el entrega dirigida de fármacos. Describe cómo la implantación del genoma viral y la replicación del virus en la célula pueden variar según el tipo de virus, pero siempre tienen el mismo resultado neto: la liberación de virus desde la célula. También se muestra cómo nanocarriers pueden superar barreras extracelulares e intracelulares para entregar fármacos a células objetivo.

Además, el documento destaca la importancia de la investigación en nanotecnología para desarrollar nuevos nanomateriales con capacidades mejoradas, que se están integrando gradualmente en aplicaciones clínicas. La FDA y la EMA han aprobado alrededor de 80 medicamentos nanotecnológicos para uso clínico.

El objetivo de las mejoras nanotecnológicas es mejorar las propiedades farmacocinéticas y farmacodinámicas de los fármacos, 

## Parte 2: Agrupación en una Clase

In [ ]:
class SimpleRAG:
    def __init__(self, model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'):
        self.embedding_model = SentenceTransformer(model_name)
        self.groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        self.open_client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTE_API_KEY")
        )
    
    def extract_text_from_pdf(self, pdf_path: str) -> str:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text

    def chunk_text(self, text: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
        chunks = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunks.append(text[start:end])
            start = end - overlap
        return chunks

    def create_embeddings(self, texts: List[str]) -> np.ndarray:
        return self.embedding_model.encode(texts)

    def _cosine_similarity(self, vec1: np.ndarray, vec2: np.ndarray) -> float:
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def semantic_search(self, query: str, chunks: List[str], chunk_embeddings: np.ndarray, k: int = 5) -> List[str]:
        query_embedding = self.create_embeddings([query])[0]
        similarity_scores = [
            (i, self._cosine_similarity(query_embedding, chunk_emb))
            for i, chunk_emb in enumerate(chunk_embeddings)
        ]
        similarity_scores.sort(key=lambda x: x[1], reverse=True)
        top_indices = [index for index, _ in similarity_scores[:k]]
        return [chunks[index] for index in top_indices]

    def _generate_llama(self, prompt_message):
        chat_completion = self.groq_client.chat.completions.create(
            messages=prompt_message,
            model="llama3-70b-8192",
        )
        return chat_completion.choices[0].message.content

    def _generate_gemini(self, prompt_message):
        model = genai.GenerativeModel("gemini-2.0-flash")
        user_prompt = prompt_message[-1]["content"]
        response = model.generate_content(user_prompt) 
        return response.text

    def _generate_qwen(self, prompt_message):
        response = self.open_client.chat.completions.create(
            model="qwen/qwen2.5-vl-72b-instruct:free",
            messages=prompt_message
        )
        return response.choices[0].message.content

    def query(self, user_query: str, context_chunks: List[str], chunk_embeddings: np.ndarray, model: str = "llama") -> str:
        relevant_context = self.semantic_search(user_query, context_chunks, chunk_embeddings)
        context_str = "\n\n".join(relevant_context)
        prompt_message = [
            {
                "role": "system",
                "content": "Eres un asistente que responde preguntas basándose únicamente en el contexto proporcionado. Si la respuesta no está en el contexto, di que no tienes suficiente información."
            },
            {
                "role": "user",
                "content": f"Contexto:\
{context_str}\\n\nPregunta: {user_query}"
            }
        ]
        if model == "llama":
            return self._generate_llama(prompt_message)
        elif model == "gemini":
            return self._generate_gemini(prompt_message)
        elif model == "qwen":
            return self._generate_qwen(prompt_message)
        else:
            raise ValueError(f"Modelo no soportado: {model}")

## Parte 3: Comparativa de Modelos

In [44]:
from IPython.display import display, Markdown

rag_pipeline = SimpleRAG()

# 1. Cargar y procesar el documento una sola vez
pdf_path = r'C:\Users\danie\Downloads\prueba\1-s2.0-S0378517324005799-main.pdf'
text = rag_pipeline.extract_text_from_pdf(pdf_path)
chunks = rag_pipeline.chunk_text(text)
embeddings = rag_pipeline.create_embeddings(chunks)

# 2. Definir la pregunta
user_query = "Realizame un resúmen del documento"

# 3. Obtener y mostrar respuestas de cada modelo
models_to_evaluate = ["llama", "gemini", "qwen"]


for model_name in models_to_evaluate:
    response = rag_pipeline.query(user_query, chunks, embeddings, model=model_name)
    display(Markdown(f"Respuesta de {model_name}: \n{response}\n\n"))

Respuesta de llama: 
El documento parece ser un artículo científico que se centra en la aplicación de soluciones nanotecnológicas para la entrega de medicamentos de manera dirigida y eficiente. A continuación, se presentan los puntos clave del documento:

* La entrega de medicamentos es un desafío en la medicina moderna, ya que los medicamentos a menudo no llegan a su destino de manera efectiva.
* En los últimos 10-20 años, la nanotecnología ha surgido como una herramienta prometedora para superar este desafío.
* Los objetivos de las mejoras nanotecnológicas incluyen:
 + Mejorar las propiedades farmacocinéticas y farmacodinámicas de los medicamentos sin alterar su estructura molecular.
 + Entregar los medicamentos de manera precisa a la estructura objetivo.
 + Superar las barreras biológicas.
 + Facilitar la producción simple y económica.
* Se han aprobado alrededor de 80 nanomedicamentos para uso clínico por la FDA y la EMA.
* El futuro de la industria farmacéutica radica en la aplicación de soluciones nanotecnológicas para mejorar los resultados terapéuticos y entregar medicamentos de manera más efectiva.
* Sin embargo, es importante abordar y superar las dificultades de traducción entre los hallazgos de los estudios en animales y su aplicación en humanos para facilitar el proceso de aprobación de medicamentos.

En general, el documento destaca la importancia de la nanotecnología en la entrega de medicamentos y su potencial para mejorar los resultados terapéuticos en la medicina moderna.



Respuesta de gemini: 
Este documento trata sobre las aplicaciones de la nanotecnología en la administración de fármacos. Destaca que el uso de nanotecnología ha aumentado en los últimos 10-20 años para la administración dirigida de fármacos, con la aprobación de varios nanomedicamentos para uso clínico. Los objetivos de las mejoras nanotecnológicas incluyen mejorar las propiedades farmacocinéticas y farmacodinámicas de los fármacos, la entrega precisa del fármaco al objetivo, sortear barreras biológicas y facilitar la producción. Además, ejemplifica el uso de nanotransportadores para la administración de fármacos, describiendo su estructura y transporte a través de barreras extracelulares e intracelulares. El documento también menciona la importancia de superar las dificultades traslacionales entre los estudios en animales y su aplicación en humanos para facilitar la aprobación de fármacos.




Respuesta de qwen: 
El documento parece ser un artículo científico que aborda el uso de nanotecnología en la entrega de medicamentos. Aquí está un resumen basado en la información proporcionada:

1. **Uso de Nanotecnología en la Entrega de Medicamentos**: El artículo destaca el aumento en el uso de soluciones nanotecnológicas para la entrega de medicamentos de manera más eficiente y precisa. Se menciona que en las últimas décadas, la investigación en este campo ha llevado al desarrollo de nuevos nanomateriales con capacidades mejoradas, que se están integrando gradualmente en aplicaciones clínicas. La FDA y la EMA han aprobado alrededor de 80 productos de nanomedicina para uso clínico.

2. **Objetivos de las Mejoras Nanotecnológicas**: Se describen los objetivos de estas mejoras, que incluyen mejorar las propiedades farmacocinéticas y farmacodinámicas de los medicamentos sin alterar su estructura molecular, entregar el medicamento de manera precisa a la estructura objetivo, superar barreras biológicas y facilitar la producción.

3. **Avances y Aplicaciones**: Se menciona un ejemplo específico donde el uso de nanotecnología ha aumentado significativamente la inhibición del crecimiento tumoral en comparación con la monoterapia. Se enfatiza el potencial de la nanotecnología para mejorar los resultados terapéuticos y la entrega de medicamentos, destacando la importancia de la investigación reciente y el enfoque de la comunidad científica en este campo.

4. **Desafíos de Traducción**: Se señala la necesidad de abordar los desafíos de traducir los hallazgos de estudios en animales a su aplicabilidad en humanos, lo cual es crucial para facilitar el proceso de aprobación de medicamentos.

5. **Contribuciones de los Autores**: Se proporciona una declaración de contribución de autoría (CRediT) que detalla las contribuciones específicas de cada autor en el artículo, como la redacción, la edición, la supervisión, la curación de datos, la conceptualización, la validación, el software, los recursos, la investigación, el análisis formal, la visualización y la administración del proyecto.

El documento parece ser una revisión o estudio que explora cómo la nanotecnología puede ser utilizada para mejorar la eficacia de los medicamentos y superar barreras biológicas, con un enfoque en la investigación y desarrollo de nuevas nanomateriales y sus aplicaciones clínicas.

